<a href="https://colab.research.google.com/github/Rchen89/demo-repo/blob/quick-test/COGS_Automation_Notebook_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="img/Adaptive-logo-long.png" width=300 align=right>

# Cost of Sales Calculation Process Automation (Step 2)

## Scope of this notebook

The objective of this notebook is to provide a Read–Eval–Print Loop (REPL) tool to support the analysis for the automation of the Cost of sales calculation process. It is important to note that this notebook doesn't replace the documentation of the analysis but rather complement it. Proving a way to quickly test our assumptions, hypothesis and experiment ideas. 

The initial section contains mainly boilerplate code. This is a temporary measure to ensure we can mount and work with files stored in Google drive. While not optimal, this approach enable us to piggyback on the built-in authentication process.

This workbook is the second step of the workbook series of Cost of Sales Calculation Process Automation. The output from the previous workbook (step 1) will be used in this workbook as the input to generate various output files


 



### *Global Utility Parameters and Functions*

#### Global Libraries

In [ ]:
## Global Libraries 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()

#### Global Parameters


In [ ]:
reporting_month_last_day = "2021-08-31" #@param {type:"date"}
calculation_period = datetime.strptime(reporting_month_last_day, "%Y-%m-%d")
calculation_period_calendar_month = calculation_period.month
calculation_period_calendar_year = calculation_period.year

adp_data_shared_drive_folder_id = '0AJ9N1TZkcrpeUk9PVA'
department_folder_id = "1wNSJVJWCyJ-DEOaopXMgUT8BztsmyxK-"

time_entries_with_cogs_google_folder_id = "1YLRZYFi6PFXeQN_iLQEdMVgKKxxn-hr6"
cogs_reference_table_google_folder_id = "139cDdEDktC28sb45F42c4sjTPzf7u-D6"
cogs_google_output_parent_folder_id = "1WZ_3TwuNSkg341j5BQWXBMJ-O8Fk8Zee"


#### Google Authentication

In [ ]:

# Install pyDrive
!pip install -U -q PyDrive2
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

mycreds_file = 'mycreds.json'

# This function authenticate pydrive instance and save the session credential 
def authenticate_pydrive():
  gauth = GoogleAuth()
  gauth.LoadCredentialsFile(mycreds_file)
  if gauth.credentials is None:
    # Authenticate if they're not there
    auth.authenticate_user()
    gauth.credentials = GoogleCredentials.get_application_default()
  elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
  else:
    # Initialize the saved creds
    gauth.Authorize()
  # Save the current credentials to a file
  gauth.SaveCredentialsFile(mycreds_file)
  
  drive = GoogleDrive(gauth)
  return drive

drive = authenticate_pydrive()

     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 3.0 MB 6.7 MB/s 


/usr/local/lib/python3.7/dist-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access mycreds.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


#### Google Drive File Handling


In [ ]:
#This function loads the most recently modified file from a google shared drive folder based on file title, file tpye and specified folder ID
def load_most_recently_modified_file_from_shared_drive_folder(file_title, file_type, parent_folder_id, retrieved_file_format, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  if file_title == "":
    file_list = drive.ListFile({'q': f"mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'", 'corpora': 'teamDrive', 'teamDriveId': adp_data_shared_drive_folder_id, 'includeItemsFromAllDrives': True, 'supportsAllDrives': True}).GetList()
  else:
    file_list = drive.ListFile({'q': f"title contains '{file_title}' and mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'", 'corpora': 'teamDrive', 'teamDriveId': adp_data_shared_drive_folder_id, 'includeItemsFromAllDrives': True, 'supportsAllDrives': True}).GetList()
  file_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("File list: ")
  for file in file_list:
    print(file["title"] + " | " + file["id"] + " | " + file["modifiedDate"] + " | " + file["mimeType"])

  if(len(file_list)>0):
    file_title = file_list[0]["title"]
    file_id = file_list[0]["id"]
    data = read_csv_from_drive(file_id,file_title)
    print("The File loaded: " + file_title)
    if retrieved_file_format == "file_name":
      return file_title
    elif retrieved_file_format == "file_id":
      return file_id
    elif retrieved_file_format == "data":
      return data
  else:
    raise NameError('No file has been found based on the current search criteria ')

#This function loads the most recently modified file from google my drive folder based on file title, file tpye and specified folder ID
def load_most_recently_modified_file_from_my_drive_folder(file_title, file_type, parent_folder_id, retrieved_file_format, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  if file_title == "":
    file_list = drive.ListFile({'q': f"mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  else:
    file_list = drive.ListFile({'q': f"title contains '{file_title}' and mimeType = '{file_type}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  file_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("File list: ")
  for file in file_list:
    print(file["title"] + " | " + file["id"] + " | " + file["modifiedDate"] + " | " + file["mimeType"])

  if(len(file_list)>0):
    file_title = file_list[0]["title"]
    file_id = file_list[0]["id"]
    data = read_csv_from_drive(file_id,file_title)
    print("The File loaded: " + file_title)
    if retrieved_file_format == "file_name":
      return file_title
    elif retrieved_file_format == "file_id":
      return file_id
    elif retrieved_file_format == "data":
      return data
  else:
    raise NameError('No file has been found based on the current search criteria ') 

#This function loads the most recently modified folder from a parent google drive folder based on the specified parent folder ID
def load_most_recently_modified_folder_from_specified_folder(parent_folder_id, last_modified_day_offset):
  last_modified_date_from_n_days_ago = (datetime.now() - timedelta(days=last_modified_day_offset)).strftime("%Y-%m-%dT%H:%M:%S")
  folder_list = drive.ListFile({'q': f"mimeType = '{'application/vnd.google-apps.folder'}' and parents in '{parent_folder_id}' and trashed=false and modifiedDate > '{last_modified_date_from_n_days_ago}'" }).GetList()
  folder_list.sort(key = lambda x : x["modifiedDate"], reverse = True)

  print("Folder list: ")
  for folder in folder_list:
    print(folder["title"] + " | " + folder["id"] + " | " + folder["modifiedDate"] + " | " + folder["mimeType"])

  if(len(folder)>0):
    folder_title = folder_list[0]["title"]
    folder_id = folder_list[0]["id"]
    print("The File loaded: " + folder_id)
    return folder_id
  else:
    raise NameError('No folder has been found based on the current search criteria ')

#This function read the csv file based on specified file id from the google drive 
def read_csv_from_drive(file_id, file_name):
    dl = drive.CreateFile({'id': file_id})
    dl.GetContentFile(file_name)
    return pd.read_csv(file_name)

#This function read the csv file based on specified file id from the google drive 
def upload_file_to_google_drive(output_file_name, file_mime_type, google_drive_folder_id, temp_filename):
    file_upload = drive.CreateFile({"title":output_file_name,
                                    "mimeType": file_mime_type, 
                                    "parents" : [{"id" : google_drive_folder_id}]})
    file_upload.SetContentFile(temp_filename)
    file_upload.Upload()
    print('Uploaded file with ID {}'.format(file_upload.get('id')))

# This function converts dataframe into csv file and upload it to specified folder in google drive
def upload_file_to_drive_csv_format(data, google_drive_folder_id, output_file_name, index_included):
    temp_filename = "data.csv"
    if index_included == True:
      data.to_csv(temp_filename,index=True)
    else:
      data.to_csv(temp_filename,index=False)
    google_drive_folder_id = google_drive_folder_id
    output_file_name = output_file_name
    upload_file_to_google_drive(output_file_name, "text/csv", google_drive_folder_id, temp_filename)



#### Data Cleasing functions


In [ ]:
# Set the data type to mulitiple columns of a dataframe
def set_column_data_type(data,col_list,data_type):
    data_copy = data.copy()
    data_copy[col_list] = data_copy[col_list].astype(data_type)
    return data_copy

#### Reference Data


##### Xero Department Code Mapping


In [ ]:
xero_department_code_mapping = load_most_recently_modified_file_from_shared_drive_folder("departments",'text/csv',department_folder_id, "data", 365)
xero_department_code_mapping.set_index("department_name", inplace = True)

File list: 
Departments.csv | 1s44BQpU31fgAqbacZtSNZfdGmMpfDncN | 2021-06-09T19:26:06.172Z | text/csv
The File loaded: Departments.csv


#### Load time entries with calculated costs dataframe

###### Find the folder which the most recently processed time entry file is stored 

In [ ]:
most_recent_cleansed_time_entry_file_folder_id = load_most_recently_modified_folder_from_specified_folder(cogs_google_output_parent_folder_id, 365)

Folder list: 
COGS_Calculation_Output_as_of_08-Sep-2021_14:44:16 | 1Bu-sm59QrVZzKnlPM1dYJVIYQ2VPTRXG | 2021-09-08T14:44:17.256Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_03-Sep-2021_10:47:28 | 1H6Whuhbg5TQZ1-jNgxIzyadI2WFPT_2f | 2021-09-03T10:47:28.586Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_10-Aug-2021_12:52:30 - Validation | 1ykXsKUJ4roH3lu2Teiw7P9Ed2uETKLgM | 2021-08-10T13:16:31.376Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_09-Aug-2021_12:11:34 | 1ege4J2ocKUe8BBIKpnA0JbAHSh7sYR7J | 2021-08-09T12:11:34.530Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_15-Jul-2021_16:00:29- For validation | 12xwyZ9jRHBij8NSEiuCaU0_rO9BGlemy | 2021-07-16T11:11:43.902Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_14-Jul-2021_20:30:46 | 1saBnrjUPKTFT7WGnhgUi-TRcyjngs3f6 | 2021-07-14T20:30:46.813Z | application/vnd.google-apps.folder
COGS_Calculation_Output_as_of_14-Jul-2021_20:27:24 

##### Helper Functions


In [ ]:
# This function retrieve the time stamp from the processed time entry file from step 1 script
def derive_output_time_stamp(file_name):
  timestamp_splitter = "|"
  index = file_name.find(timestamp_splitter)
  return file_name[index+1:len(file_name)-4]

# This function derives the department code for AI template
def derive_department_code_for_AI_template(row):
  business_unit_mapping_for_AI = {
      "Adaptive Canada" : "MTL",
      "Adaptive UK" : "LDN",
      "Adaptive US" : "NY",
      "Adaptive Spain" : "BCN"
  }
  resource_BU = row["resource_BU"]
  adp_department = row["adp_department"]
  
  return adp_department + " - " + business_unit_mapping_for_AI[resource_BU]

# This function derives the reference column in the output file 
def derive_reference_column(row):
  resourced_activity = row["resourced_activity"]
  delivery_element_reference = row["delivery_element_reference"]
  if delivery_element_reference != "n/a":
    return delivery_element_reference
  else:
    return resourced_activity

# This function derives the tax rate column on the journal entry template
def derive_journal_tax_rate(row):
  return "Tax Exempt"

# This function filters the full journal entry dataframe by each business unit, then upload the resulting dataframe as csv file to target google drive folder
def process_cogs_journal_entry_by_BU(df_journal_entris_all, business_unit, output_batch_code):
  cogs_journal_template_final_BU = cogs_journal_template_final[cogs_journal_template_final["Entity"] == business_unit]
  cogs_journal_template_file_name = "COGS - Journal Template Output_" + business_unit + " | " + output_time_stamp + ".csv"
  upload_file_to_drive_csv_format(cogs_journal_template_final_BU, most_recent_cleansed_time_entry_file_folder_id, cogs_journal_template_file_name, False)
  return cogs_journal_template_final_BU
    
# This function derives the line description for B/S reconciliation file    
def derive_bs_acc_line_description(row):
  entry_units_by_day = row["entry_units_by_day"]
  daily_rate = row["bamboohr_cost_rate_daily"]
  resource = row["resource"]

  return str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year) + " " + resource + " " + str(round(entry_units_by_day, 3)) + " days at " + str(daily_rate)

#### Load the most recently processed time entry file with calculated costs

In [ ]:
time_entries_with_calculated_cost_file_name = load_most_recently_modified_file_from_my_drive_folder('COGS_Output_2_Kimble_Time_Entries', 'text/csv', most_recent_cleansed_time_entry_file_folder_id, "file_name", 30)
output_time_stamp =  derive_output_time_stamp(time_entries_with_calculated_cost_file_name)
time_entries_with_calculated_cost = load_most_recently_modified_file_from_my_drive_folder('COGS_Output_2_Kimble_Time_Entries', 'text/csv', most_recent_cleansed_time_entry_file_folder_id, "data", 30)
time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["cost_GL"].notnull(), "cost_GL"] = time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["cost_GL"].notnull(), "cost_GL"].astype(int).astype(str)
time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["contra_GL"].notnull(), "contra_GL"] = time_entries_with_calculated_cost.loc[time_entries_with_calculated_cost["contra_GL"].notnull(), "contra_GL"].astype(int).astype(str)
time_entries_with_calculated_cost["reference"] = time_entries_with_calculated_cost.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_with_calculated_cost.fillna("n/a",inplace = True)
time_entries_with_calculated_cost.head()

File list: 
COGS_Output_2_Kimble_Time_Entries | 08-Sep-2021_14:44:16.csv | 1KVaWd_Demauxv7EIS7fx_SAAoM0omThP | 2021-09-08T14:44:41.921Z | text/csv
The File loaded: COGS_Output_2_Kimble_Time_Entries | 08-Sep-2021_14:44:16.csv
File list: 
COGS_Output_2_Kimble_Time_Entries | 08-Sep-2021_14:44:16.csv | 1KVaWd_Demauxv7EIS7fx_SAAoM0omThP | 2021-09-08T14:44:41.921Z | text/csv
The File loaded: COGS_Output_2_Kimble_Time_Entries | 08-Sep-2021_14:44:16.csv


,Id,timesheet_name,activity_assignment,resourced_activity,is_resourced_activity_unpaid,account,delivery_engagement,delivery_element_reference,delivery_element_shortname,product,resource,resource_id,resource_type,resource_BU,resource_BU_currency,resource_department,time_period,date,period_rate_band,kimble_cost_rate_hourly,cost_rate_currency,use_kimble_cost_rate,cost_rate_factor,entry_units,adp_department,adp_cost_code,adp_revenue_cost_category,entry_units_by_day,bamboohr_cost_rate_daily,currency_exchange_rate,kimble_cost_rate_hourly_resource_currency,cost_for_activity,cost_GL,cost_GL_name,contra_GL,contra_GL_name,reference
0,a4T4J000001WkJBUA0,Adaptive US-Finance (US) (KE0000163-001) - Mic...,"Michelle Kriel, on Adaptive US, (01/04/2020 -...",Finance (US) (KE0000163-001),False,Adaptive US,Finance (US),KE0000163-001,Finance (US),Internal/Non-billable Resource (cost),Michelle Kriel,40733,Employee,Adaptive UK,GBP,Finance,Tue 1,2021-06-01,Business Day - Standard Time,108.3585,USD,False,1.0,0.5,Finance,Administration,n/a,0.0625,580.075758,0.7056,76.457758,36.254735,n/a,n/a,n/a,n/a,KE0000163-001
1,a4T4J000001WkJOUA0,Adaptive Global-Finance (KE0000112-001) - Mich...,"Michelle Kriel, on Adaptive Global, (01/04/202...",Finance (KE0000112-001),False,Adaptive Global,Finance,KE0000112-001,Finance,Internal/Non-billable Resource (cost),Michelle Kriel,40733,Employee,Adaptive UK,GBP,Finance,Tue 1,2021-06-01,Business Day - Standard Time,80.6250,GBP,False,1.0,8.0,Finance,Administration,n/a,1.0000,580.075758,1.0000,80.625000,580.075758,n/a,n/a,n/a,n/a,KE0000112-001
2,a4T4J000001WkswUAC,BP International Limited-BP- LNG Extension 202...,"Leon Coto-Rodriguez, on BP International Limit...",BP- LNG Extension 2021 (KE0000215-002),False,BP International Limited,BP- LNG Extension 2021,KE0000215-002,BP- LNG Extension 2021,CONSULTING - Build - FP (Cost%Complete),Leon Coto-Rodriguez,40710,Employee,Adaptive UK,GBP,Consulting,Tue 1,2021-06-01,Business Day - Standard Time,87.0000,GBP,False,1.0,8.0,Consulting,Delivery,General Consulting,1.0000,580.075758,1.0000,87.000000,580.075758,5015,Client billable permanent resource,5400,Consulting Permanent remuneration,KE0000215-002
3,a4T4J000001WksxUAC,BP International Limited-BP- LNG Extension 202...,"Leon Coto-Rodriguez, on BP International Limit...",BP- LNG Extension 2021 (KE0000215-002),False,BP International Limited,BP- LNG Extension 2021,KE0000215-002,BP- LNG Extension 2021,CONSULTING - Build - FP (Cost%Complete),Leon Coto-Rodriguez,40710,Employee,Adaptive UK,GBP,Consulting,Wed 2,2021-06-02,Business Day - Standard Time,87.0000,GBP,False,1.0,8.0,Consulting,Delivery,General Consulting,1.0000,580.075758,1.0000,87.000000,580.075758,5015,Client billable permanent resource,5400,Consulting Permanent remuneration,KE0000215-002
4,a4T4J000001WksyUAC,BP International Limited-BP- LNG Extension 202...,"Leon Coto-Rodriguez, on BP International Limit...",BP- LNG Extension 2021 (KE0000215-002),False,BP International Limited,BP- LNG Extension 2021,KE0000215-002,BP- LNG Extension 2021,CONSULTING - Build - FP (Cost%Complete),Leon Coto-Rodriguez,40710,Employee,Adaptive UK,GBP,Consulting,Thu 3,2021-06-03,Business Day - Standard Time,87.0000,GBP,False,1.0,8.0,Consulting,Delivery,General Consulting,1.0000,580.075758,1.0000,87.000000,580.075758,5015,Client billable permanent resource,5400,Consulting Permanent remuneration,KE0000215-002


#### Generate AI Upload Template 


In [ ]:
time_entries_pivot_table_for_AI_template = pd.pivot_table(time_entries_with_calculated_cost, index=["resource_BU", "resource", "resource_id", "resource_type", "cost_GL", "resourced_activity", "adp_department", "delivery_element_reference", "adp_revenue_cost_category"], values = ["cost_for_activity", "entry_units_by_day"], aggfunc="sum")

time_entries_pivot_table_for_AI_template

cost_for_activity  entry_units_by_day
resource_BU     resource        resource_id resource_type cost_GL resourced_activity                                 adp_department delivery_element_reference adp_revenue_cost_category                                       
Adaptive Canada Adam Smith      40933       Employee      5100    Available Capacity (Canada) (KE0000152-001)        Consulting     KE0000152-001              Available Capacity              10218.545516             17.0000
                Anthony Dahanne 40834       Employee      5015    NBC-Retail-Jun2020-Jun2021 (API) (KE0000038-005)   Consulting     KE0000038-005              General Consulting               5595.331808              8.4375
                                                                  NBC-Retail-Nov2019-Nov2021 Extra Capacity (KE00... Consulting     KE0000038-010              Available Capacity                373.022121              0.5625
                Anton Milanov   40585       Employee      5015    BNYM - Primerica - Phase D (CAD) (KE0000332-002)   Consulting     KE0000332-002              General Consulting               8175.566548             19.0000
                                                          5101    Sickness/Short Term Absence                        Consulting     n/a                        Vacation and absence              860.585952              2.0000
...                                                                                                                                                                                                     ...                 ...
Adaptive US     Timothy Meehan  40790       Employee      5015    BNYM - Active Advisor - UX (KE0000384-001)         Consulting     KE0000384-001              General Consulting              16792.729545             21.0000
                                                          5101    Sickness/Short Term Absence                        Consulting     n/a                        Vacation and absence              799.653788              1.0000
                Tracy Tseung    40925       Employee      5015    DriveWealth - Fractional Trading Platform - Bui... Consulting     KE0000305-001              Accelerated Solution            17646.133333             22.0000
                Travis Zerba    40928       Employee      5015    BNYM - Sumday - Team Eliza [USD] (KE0000335-001)   Consulting     KE0000335-001              General Consulting               8241.754545             12.0000
                                                          5100    Available Capacity (US) (KE0000151-001)            Consulting     KE0000151-001              Available Capacity               6868.128788             10.0000

[505 rows x 2 columns]

In [ ]:
time_entries_pivot_table_flattened_for_AI_template = pd.DataFrame(time_entries_pivot_table_for_AI_template.to_records())
time_entries_pivot_table_flattened_for_AI_template["department_code_AI"] = time_entries_pivot_table_flattened_for_AI_template.apply(lambda row: derive_department_code_for_AI_template(row), axis=1)
time_entries_pivot_table_flattened_for_AI_template["reference"] = time_entries_pivot_table_flattened_for_AI_template.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_pivot_table_flattened_for_AI_template = time_entries_pivot_table_flattened_for_AI_template[time_entries_pivot_table_flattened_for_AI_template["cost_GL"].str.startswith("5")]
column_order = ["department_code_AI", "resource", "resource_id", "resource_type", "cost_for_activity", "entry_units_by_day", "reference", "adp_revenue_cost_category", "adp_department"]
time_entries_pivot_table_flattened_for_AI_template = time_entries_pivot_table_flattened_for_AI_template[column_order]
time_entries_pivot_table_flattened_for_AI_template

cogs_AI_template_file_name = "COGS - AI Template Output" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template, most_recent_cleansed_time_entry_file_folder_id, cogs_AI_template_file_name, False)

Uploaded file with ID 1oeOJbLqNRDCa76bd2fNNBLq4XSOC3Ftx


In [ ]:
time_entries_pivot_table_flattened_for_AI_template_by_days = time_entries_pivot_table_flattened_for_AI_template
column_order_by_days =  ["department_code_AI", "resource", "resource_id", "resource_type", "reference", "adp_department", "entry_units_by_day"]
time_entries_pivot_table_flattened_for_AI_template_by_days = time_entries_pivot_table_flattened_for_AI_template_by_days[column_order_by_days]

cogs_by_days_AI_template_file_name = "COGS - AI Template Output - by days" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template_by_days, most_recent_cleansed_time_entry_file_folder_id, cogs_by_days_AI_template_file_name, False)


time_entries_pivot_table_flattened_for_AI_template_by_cost = time_entries_pivot_table_flattened_for_AI_template
column_order_by_costs = ["department_code_AI", "resource", "resource_id", "reference", "resource_type", "adp_department", "cost_for_activity"]
time_entries_pivot_table_flattened_for_AI_template_by_cost = time_entries_pivot_table_flattened_for_AI_template_by_cost[column_order_by_costs]

cogs_by_costs_AI_template_file_name = "COGS - AI Template Output - by costs" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_AI_template_by_cost, most_recent_cleansed_time_entry_file_folder_id, cogs_by_costs_AI_template_file_name, False)


Uploaded file with ID 15LfRT-ajx0Z2JhSPPwUJpwAhqAwxjjR0
Uploaded file with ID 1yJy8H9K5bujetPVvVXxm2KRn5M0Jii2i


#### Generate Journal Entry Upload Template 

In [ ]:
time_entries_pivot_table_for_journal_template = pd.pivot_table(time_entries_with_calculated_cost, index=["resource_BU", "cost_GL", "adp_department", "contra_GL", "resource_department", "resourced_activity", "delivery_element_reference"], values = ["cost_for_activity"], aggfunc="sum")
time_entries_pivot_table_for_journal_template

cost_for_activity
resource_BU     cost_GL adp_department contra_GL resource_department resourced_activity                                 delivery_element_reference                   
Adaptive Canada 5010    Consulting     2111      Consulting          BNYM - Primerica - Phase D (CAD) (KE0000332-002)   KE0000332-002                    12920.000000
                                                                     BNYM - Secure Choice Retirement Portal [CAD]) (... KE0000333-002                    15700.000000
                                                                     BNYM - Sumday - Team Eliza [CAD] (KE0000335-002)   KE0000335-002                    14900.000000
                5015    Consulting     5400      Consulting          BNYM - Future of Collateral 2021 (CAD) (KE00002... KE0000277-003                    19804.428373
                                                                     BNYM - Primerica - Phase D (CAD) (KE0000332-002)   KE0000332-002                    18476.500238
...                                                                                                                                                               ...
Adaptive US     n/a     Management     n/a       Management          Office Management (US) (KE0000132-001)             KE0000132-001                     9021.551136
                                                                     Pre-sales (KE0000133-002)                          KE0000133-002                     1954.669413
                                                                     Prospecting/Lead generation (KE0000133-001)        KE0000133-001                      751.795928
                        Sales          n/a       Sales               General Sales Activities (KE0000133-003)           KE0000133-003                    18861.507955
                                                                     Sickness/Short Term Absence                        n/a                                898.167045

[264 rows x 1 columns]

In [ ]:
# time_entries_pivot_table_flattened_for_journal_template = pd.DataFrame(time_entries_pivot_table_for_journal_template.to_records())
# upload_file_to_drive_csv_format(time_entries_pivot_table_flattened_for_journal_template, most_recent_cleansed_time_entry_file_folder_id, "test_journal_pivot_data.csv", False)

In [ ]:
time_entries_pivot_table_flattened_for_journal_template = pd.DataFrame(time_entries_pivot_table_for_journal_template.to_records())
time_entries_pivot_table_flattened_for_journal_template = time_entries_pivot_table_flattened_for_journal_template[time_entries_pivot_table_flattened_for_journal_template["cost_GL"] != "n/a"]
time_entries_pivot_table_flattened_for_journal_template["cost_for_activity"] = time_entries_pivot_table_flattened_for_journal_template["cost_for_activity"].round(2) 
time_entries_pivot_table_flattened_for_journal_template["Entity"] = time_entries_pivot_table_flattened_for_journal_template["resource_BU"]
time_entries_pivot_table_flattened_for_journal_template["Date"] = reporting_month_last_day
time_entries_pivot_table_flattened_for_journal_template["Description"] = "COGS " + str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year)
time_entries_pivot_table_flattened_for_journal_template["Narration"] = "COGS " + str(calculation_period_calendar_month) + "-" + str(calculation_period_calendar_year) + " - " + "See Cogs files for back up"
time_entries_pivot_table_flattened_for_journal_template["Tax Rate"] = time_entries_pivot_table_flattened_for_journal_template.apply(lambda row: derive_journal_tax_rate(row), axis = 1)
time_entries_pivot_table_flattened_for_journal_template["TrackingOption1"] = time_entries_pivot_table_flattened_for_journal_template["adp_department"].map(xero_department_code_mapping["xero_department_name"])
time_entries_pivot_table_flattened_for_journal_template["TrackingOption2"] = time_entries_pivot_table_flattened_for_journal_template.apply(lambda row: derive_reference_column(row), axis=1)
time_entries_pivot_table_flattened_for_journal_template

,resource_BU,cost_GL,adp_department,contra_GL,resource_department,resourced_activity,delivery_element_reference,cost_for_activity,Entity,Date,Description,Narration,Tax Rate,TrackingOption1,TrackingOption2
0,Adaptive Canada,5010,Consulting,2111,Consulting,BNYM - Primerica - Phase D (CAD) (KE0000332-002),KE0000332-002,12920.00,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000332-002
1,Adaptive Canada,5010,Consulting,2111,Consulting,BNYM - Secure Choice Retirement Portal [CAD]) ...,KE0000333-002,15700.00,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000333-002
2,Adaptive Canada,5010,Consulting,2111,Consulting,BNYM - Sumday - Team Eliza [CAD] (KE0000335-002),KE0000335-002,14900.00,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000335-002
3,Adaptive Canada,5015,Consulting,5400,Consulting,BNYM - Future of Collateral 2021 (CAD) (KE0000...,KE0000277-003,19804.43,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000277-003
4,Adaptive Canada,5015,Consulting,5400,Consulting,BNYM - Primerica - Phase D (CAD) (KE0000332-002),KE0000332-002,18476.50,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000332-002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Adaptive US,5101,Consulting,5400,Consulting,US Vacation Starting 2021,n/a,45405.60,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,US Vacation Starting 2021
247,Adaptive US,5201,CBU,5400,CBU,Account and Portfolio Management (US) (KE00001...,KE0000126-001,12028.73,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,90 Central Business Unit (CBU),KE0000126-001
248,Adaptive US,5203,Consulting,5400,Consulting,Graduate Programme 2021 (KE0000146-002),KE0000146-002,19817.39,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,100 Consulting,KE0000146-002
249,Adaptive US,6016,Operate,6020,Operate,Operate Management (KE0000107-003),KE0000107-003,11074.86,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,Tax Exempt,120 Operate,KE0000107-003


In [ ]:
cogs_journal_template_debit_lines = time_entries_pivot_table_flattened_for_journal_template.copy()
cogs_journal_template_debit_lines["Accounting Code"] = cogs_journal_template_debit_lines["cost_GL"]
cogs_journal_template_debit_lines["Amount"] = cogs_journal_template_debit_lines["cost_for_activity"]
journal_columns = ["Entity", "Date", "Description", "Narration", "Accounting Code", "Tax Rate", "Amount", "TrackingOption1", "TrackingOption2"]
cogs_journal_template_debit_lines = cogs_journal_template_debit_lines[journal_columns]
cogs_journal_template_debit_lines

,Entity,Date,Description,Narration,Accounting Code,Tax Rate,Amount,TrackingOption1,TrackingOption2
0,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,12920.00,100 Consulting,KE0000332-002
1,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,15700.00,100 Consulting,KE0000333-002
2,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,14900.00,100 Consulting,KE0000335-002
3,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5015,Tax Exempt,19804.43,100 Consulting,KE0000277-003
4,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5015,Tax Exempt,18476.50,100 Consulting,KE0000332-002
...,...,...,...,...,...,...,...,...,...
246,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5101,Tax Exempt,45405.60,100 Consulting,US Vacation Starting 2021
247,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5201,Tax Exempt,12028.73,90 Central Business Unit (CBU),KE0000126-001
248,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5203,Tax Exempt,19817.39,100 Consulting,KE0000146-002
249,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,6016,Tax Exempt,11074.86,120 Operate,KE0000107-003


In [ ]:
cogs_journal_template_credit_lines = time_entries_pivot_table_flattened_for_journal_template.copy()
cogs_journal_template_credit_lines["Accounting Code"] = cogs_journal_template_credit_lines["contra_GL"]
cogs_journal_template_credit_lines["Amount"] = cogs_journal_template_credit_lines["cost_for_activity"] * -1
journal_columns = ["Entity", "Date", "Description", "Narration", "Accounting Code", "Tax Rate", "Amount", "TrackingOption1", "TrackingOption2"]
cogs_journal_template_credit_lines = cogs_journal_template_credit_lines[journal_columns]
cogs_journal_template_credit_lines

,Entity,Date,Description,Narration,Accounting Code,Tax Rate,Amount,TrackingOption1,TrackingOption2
0,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,2111,Tax Exempt,-12920.00,100 Consulting,KE0000332-002
1,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,2111,Tax Exempt,-15700.00,100 Consulting,KE0000333-002
2,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,2111,Tax Exempt,-14900.00,100 Consulting,KE0000335-002
3,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-19804.43,100 Consulting,KE0000277-003
4,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-18476.50,100 Consulting,KE0000332-002
...,...,...,...,...,...,...,...,...,...
246,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-45405.60,100 Consulting,US Vacation Starting 2021
247,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-12028.73,90 Central Business Unit (CBU),KE0000126-001
248,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-19817.39,100 Consulting,KE0000146-002
249,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,6020,Tax Exempt,-11074.86,120 Operate,KE0000107-003


In [ ]:
cogs_journal_template_final = pd.concat([cogs_journal_template_debit_lines, cogs_journal_template_credit_lines],axis = 0)
cogs_journal_template_final

,Entity,Date,Description,Narration,Accounting Code,Tax Rate,Amount,TrackingOption1,TrackingOption2
0,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,12920.00,100 Consulting,KE0000332-002
1,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,15700.00,100 Consulting,KE0000333-002
2,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5010,Tax Exempt,14900.00,100 Consulting,KE0000335-002
3,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5015,Tax Exempt,19804.43,100 Consulting,KE0000277-003
4,Adaptive Canada,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5015,Tax Exempt,18476.50,100 Consulting,KE0000332-002
...,...,...,...,...,...,...,...,...,...
246,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-45405.60,100 Consulting,US Vacation Starting 2021
247,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-12028.73,90 Central Business Unit (CBU),KE0000126-001
248,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,5400,Tax Exempt,-19817.39,100 Consulting,KE0000146-002
249,Adaptive US,2021-08-31,COGS 8-2021,COGS 8-2021 - See Cogs files for back up,6020,Tax Exempt,-11074.86,120 Operate,KE0000107-003


In [ ]:
cogs_journal_template_final_uk = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive UK", output_time_stamp)
cogs_journal_template_final_us = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive US", output_time_stamp)
cogs_journal_template_final_spain = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive Spain", output_time_stamp)
cogs_journal_template_final_canada = process_cogs_journal_entry_by_BU(cogs_journal_template_final, "Adaptive Canada", output_time_stamp)

Uploaded file with ID 1iPBuSwac1plfZ5z6sbIY4SGkXrl-j14g
Uploaded file with ID 1Fza90HLgyrw4G4nCeEm6D_PcWemh9bEW
Uploaded file with ID 1BWLAtFCI3Z7qUXpWyUJawllSkgePPFpx
Uploaded file with ID 1s6XmwgppbMsWkGaIKdOY3LhWLtU4CRZy


##### Contractor BS rec acc LC

In [ ]:
time_entries_with_calculated_cost_contractors = time_entries_with_calculated_cost[time_entries_with_calculated_cost["resource_type"] == "Contractor"]
time_entries_with_calculated_cost_contractors

,Id,timesheet_name,activity_assignment,resourced_activity,is_resourced_activity_unpaid,account,delivery_engagement,delivery_element_reference,delivery_element_shortname,product,resource,resource_id,resource_type,resource_BU,resource_BU_currency,resource_department,time_period,date,period_rate_band,kimble_cost_rate_hourly,cost_rate_currency,use_kimble_cost_rate,cost_rate_factor,entry_units,adp_department,adp_cost_code,adp_revenue_cost_category,entry_units_by_day,bamboohr_cost_rate_daily,currency_exchange_rate,kimble_cost_rate_hourly_resource_currency,cost_for_activity,cost_GL,cost_GL_name,contra_GL,contra_GL_name,reference
24,a4T4J000001WlJqUAK,The Bank of New York Mellon-BNYM - Sumday - Te...,"Leonardo Ferreira de Moura, on The Bank of New...",BNYM - Sumday - Team Eliza [CAD] (KE0000335-002),False,The Bank of New York Mellon,BNYM - Sumday - Team Eliza,KE0000335-002,BNYM - Sumday - Team Eliza [CAD],CONSULTING - Build - T&M (daily),Leonardo Ferreira de Moura,40914,Contractor,Adaptive Canada,CAD,Consulting,Tue 1,2021-06-01,Business Day - Standard Time,79.25,USD,False,1.0,8.0,Consulting,Delivery,General Consulting,1.00,800.0,1.2019,95.250575,800.0,5010,Consultancy resources,2111,Contractor accruals,KE0000335-002
25,a4T4J000001WlJvUAK,The Bank of New York Mellon-BNYM - Sumday - Te...,"Leonardo Ferreira de Moura, on The Bank of New...",BNYM - Sumday - Team Eliza [CAD] (KE0000335-002),False,The Bank of New York Mellon,BNYM - Sumday - Team Eliza,KE0000335-002,BNYM - Sumday - Team Eliza [CAD],CONSULTING - Build - T&M (daily),Leonardo Ferreira de Moura,40914,Contractor,Adaptive Canada,CAD,Consulting,Wed 2,2021-06-02,Business Day - Standard Time,79.25,USD,False,1.0,8.0,Consulting,Delivery,General Consulting,1.00,800.0,1.2062,95.591350,800.0,5010,Consultancy resources,2111,Contractor accruals,KE0000335-002
39,a4T4J000001WlNeUAK,ADS Securities LLC-ADSS- Phoenix Day 2 London ...,"Vivek Ahuja, on ADS Securities LLC, (4/05/2021...",ADSS- Phoenix Day 2 London MVP (KE0000242-003),False,ADS Securities LLC,ADSS- Phoenix Day 2 London MVP,KE0000242-003,ADSS- Phoenix Day 2 London MVP,CONSULTING - Accelerated Build - T&M (daily),Vivek Ahuja,40620,Contractor,Adaptive UK,GBP,Consulting,Tue 1,2021-06-01,Business Day - Standard Time,91.75,GBP,False,1.0,8.0,Consulting,Delivery,Accelerated Solution,1.00,734.0,1.0000,91.750000,734.0,5010,Consultancy resources,2111,Contractor accruals,KE0000242-003
40,a4T4J000001WlNfUAK,ADS Securities LLC-ADSS- Phoenix Day 2 London ...,"Vivek Ahuja, on ADS Securities LLC, (4/05/2021...",ADSS- Phoenix Day 2 London MVP (KE0000242-003),False,ADS Securities LLC,ADSS- Phoenix Day 2 London MVP,KE0000242-003,ADSS- Phoenix Day 2 London MVP,CONSULTING - Accelerated Build - T&M (daily),Vivek Ahuja,40620,Contractor,Adaptive UK,GBP,Consulting,Wed 2,2021-06-02,Business Day - Standard Time,91.75,GBP,False,1.0,8.0,Consulting,Delivery,Accelerated Solution,1.00,734.0,1.0000,91.750000,734.0,5010,Consultancy resources,2111,Contractor accruals,KE0000242-003
41,a4T4J000001WlNgUAK,ADS Securities LLC-ADSS- Phoenix Day 2 London ...,"Vivek Ahuja, on ADS Securities LLC, (4/05/2021...",ADSS- Phoenix Day 2 London MVP (KE0000242-003),False,ADS Securities LLC,ADSS- Phoenix Day 2 London MVP,KE0000242-003,ADSS- Phoenix Day 2 London MVP,CONSULTING - Accelerated Build - T&M (daily),Vivek Ahuja,40620,Contractor,Adaptive UK,GBP,Consulting,Thu 3,2021-06-03,Business Day - Standard Time,91.75,GBP,False,1.0,8.0,Consulting,Delivery,Accelerated Solution,1.00,734.0,1.0000,91.750000,734.0,5010,Consultancy resources,2111,Contractor accruals,KE0000242-003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5726,a4T4J000002gasWUAQ,The Bank of New York Mellon-BNYM - Sumday - Te...,"Leonardo Ferreira de Moura, on The Bank of New...",BNYM - Sumday - Team Eliza [CAD] (KE0000335-002),False,The Bank of New York Mellon,BNYM - Sumday - Team Eliza,KE0000335-002,BNYM - Sumday - Team

In [ ]:
time_entries_pivot_for_bs_acc_contractor = pd.pivot_table(time_entries_with_calculated_cost_contractors, index=["resource_BU", "contra_GL", "resource", "bamboohr_cost_rate_daily", "delivery_element_shortname", "delivery_element_reference", "adp_department"], values = ["entry_units_by_day", "cost_for_activity"], aggfunc="sum")
time_entries_pivot_for_bs_acc_contractor

cost_for_activity  entry_units_by_day
resource_BU     contra_GL resource                   bamboohr_cost_rate_daily delivery_element_shortname                       delivery_element_reference adp_department                                       
Adaptive Canada 2111      Leonardo Ferreira de Moura 800.0                    BNYM - Sumday - Team Eliza [CAD]                 KE0000335-002              Consulting              14900.000              18.625
                                                                              n/a                                              n/a                        Consulting                  0.000               2.375
                          Louis Laramée              800.0                    BNYM - Secure Choice Retirement Portal [CAD])    KE0000333-002              Consulting              15700.000              19.625
                                                                              n/a                                              n/a                        Consulting                  0.000               1.375
                          Viktor Rospotniuk          760.0                    BNYM - Primerica - Phase D (CAD)                 KE0000332-002              Consulting              12920.000              17.000
...                                                                                                                                                                                     ...                 ...
Adaptive UK     2111      Tim Kimber                 675.0                    n/a                                              n/a                        Consulting                  0.000               4.000
                          Vivek Ahuja                734.0                    ADSS- Phoenix Day 2 London MVP                   KE0000242-003              Consulting              16148.000              22.000
                          Yip Wong                   750.0                    ADSS- Phoenix Day 2 London MVP                   KE0000242-003              Consulting              16125.000              21.500
                                                                              n/a                                              n/a                        Consulting                  0.000               0.500
                          Zeeshan Pervez             675.0                    EBS Group - BrokerTec Quote 2021 (Jun-Dec) [GBP] KE0000357-002              Consulting              15440.625              22.750

[66 rows x 2 columns]

In [ ]:
time_entries_pivot_flatterned_for_bs_acc_contractor = pd.DataFrame(time_entries_pivot_for_bs_acc_contractor.to_records())
time_entries_pivot_flatterned_for_bs_acc_contractor["description"] = time_entries_pivot_flatterned_for_bs_acc_contractor.apply(lambda row: derive_bs_acc_line_description(row), axis = 1)
time_entries_pivot_flatterned_for_bs_acc_contractor["cost_on_bs"] = time_entries_pivot_flatterned_for_bs_acc_contractor["cost_for_activity"] * -1
time_entries_pivot_flatterned_for_bs_acc_contractor = time_entries_pivot_flatterned_for_bs_acc_contractor[time_entries_pivot_flatterned_for_bs_acc_contractor["cost_on_bs"] != 0]
ordered_columns = ["resource_BU", "resource", "bamboohr_cost_rate_daily", "entry_units_by_day", "cost_for_activity", "cost_on_bs", "description", "delivery_element_reference", "delivery_element_shortname", "adp_department"]
time_entries_pivot_flatterned_for_bs_acc_contractor = time_entries_pivot_flatterned_for_bs_acc_contractor[ordered_columns]
time_entries_pivot_flatterned_for_bs_acc_contractor


,resource_BU,resource,bamboohr_cost_rate_daily,entry_units_by_day,cost_for_activity,cost_on_bs,description,delivery_element_reference,delivery_element_shortname,adp_department
0,Adaptive Canada,Leonardo Ferreira de Moura,800.0,18.6250,14900.00000,-14900.00000,8-2021 Leonardo Ferreira de Moura 18.625 days ...,KE0000335-002,BNYM - Sumday - Team Eliza [CAD],Consulting
2,Adaptive Canada,Louis Laramée,800.0,19.6250,15700.00000,-15700.00000,8-2021 Louis Laramée 19.625 days at 800.0,KE0000333-002,BNYM - Secure Choice Retirement Portal [CAD]),Consulting
4,Adaptive Canada,Viktor Rospotniuk,760.0,17.0000,12920.00000,-12920.00000,8-2021 Viktor Rospotniuk 17.0 days at 760.0,KE0000332-002,BNYM - Primerica - Phase D (CAD),Consulting
6,Adaptive UK,Alex Trull,700.0,10.8750,7612.50000,-7612.50000,8-2021 Alex Trull 10.875 days at 700.0,KE0000240-001,ADSS- Phoenix Infra Build [GBP],Operate
8,Adaptive UK,Andres Gutierrez,715.0,2.1875,1564.06250,-1564.06250,8-2021 Andres Gutierrez 2.188 days at 715.0,KE0000146-002,Graduate Programme 2021,Consulting
9,Adaptive UK,Andres Gutierrez,715.0,2.1875,1564.06250,-1564.06250,8-2021 Andres Gutierrez 2.188 days at 715.0,KE0000147-002,Graduate Programme 2021,Consulting
10,Adaptive UK,Andres Gutierrez,715.0,7.1250,5094.37500,-5094.37500,8-2021 Andres Gutierrez 7.125 days at 715.0,KE0000143-002,Graduate Programme Core 2021-22 (Global),Consulting
11,Adaptive UK,Andres Gutierrez,715.0,10.5000,7507.50000,-7507.50000,8-2021 Andres Gutierrez 10.5 days at 715.0,KE0000128-001,Reactive Trader Management (core),Consulting
12,Adaptive UK,Andrzej Gladkowski,750.0,18.0000,13500.00000,-13500.00000,8-2021 Andrzej Gladkowski 18.0 days at 750.0,KE0000242-003,ADSS- Phoenix Day 2 London MVP,Consulting
14,Adaptive UK,Antonius Elmo,450.0,21.0000,9450.00000,-9450.00000,8-2021 Antonius Elmo 21.0 days at 450.0,KE0000242-003,ADSS- Phoenix Day 2 London MVP,Consulting


In [ ]:
cogs_bs_acc_file_name = "COGS - B/S acc Output" + " | " + output_time_stamp + ".csv"
upload_file_to_drive_csv_format(time_entries_pivot_flatterned_for_bs_acc_contractor, most_recent_cleansed_time_entry_file_folder_id, cogs_bs_acc_file_name, False)

Uploaded file with ID 1YvqBn1wYJKW42Md4CJ5XenqarsFAueaM
